# Analyzing borrowers’ risk of defaulting
The purposes of the project and hypotheses in General is need to find out if a customer’s marital status and number of children have an impact on whether they will default on a loan. the following objectives will be performed:
1. Open the data file /datasets/credit_scoring_eng.csv
2. Preprocess the data:
- Identify and fill in missing values
- Replace the real number data type with the integer type
- Delete duplicate data
- Categorize the data
3. Answer these questions:
- Is there a connection between having kids and repaying a loan on time?
- Is there a connection between marital status and repaying a loan on time?
- Is there a connection between income level and repaying a loan on time?
- How do different loan purposes affect on-time loan repayment?
4. Write an overall conclusion.

## Loading data 

In [142]:
# Loading all the libraries
import pandas as pd
import numpy as np

#to output more than 1 variable
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Load the data
path = '/Volumes/KINGSTON/Docs/Rovner/Israel/Course Data Analyst/Project2//'

try:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')
except FileNotFoundError:
    df = pd.read_csv(path+'credit_scoring_eng.csv')

## Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan



In [143]:
# Let's see how many rows and columns our dataset has
rows=len(df.index)
columns=len(df.columns) 
print(rows, columns)

21525 12


In [144]:
# let's print the first N rows
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [145]:
# Get info on data
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [146]:
# Let's look in the filtered table at the the first column with missing data
df['days_employed'].isna() 

0        False
1        False
2        False
3        False
4        False
         ...  
21520    False
21521    False
21522    False
21523    False
21524    False
Name: days_employed, Length: 21525, dtype: bool

Preliminary it seems that it is, it looks symmetrical. But it is necessary to conduct an additional search for duplicates and missing values.                           

In [147]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

The number of rows match the rows of  missing values. Need to see detailed duplicates and missing values for each rows of mentioned columns. To do this, will use unique(), isna() and duplicates() methods.                            

In [148]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
days_employed=df['days_employed'].value_counts(dropna=False)
total_income=df['total_income'].value_counts(dropna=False) 
print(days_employed)
print(total_income)

 NaN              2174
-8437.673028         1
-5135.928528         1
 354500.415854       1
-769.717438          1
                  ... 
-1099.957609         1
-209.984794          1
 398099.392433       1
-1271.038880         1
-1984.507589         1
Name: days_employed, Length: 19352, dtype: int64
NaN          2174
17312.717       2
42413.096       2
31791.384       2
21005.772       1
             ... 
27020.895       1
23686.835       1
9606.294        1
28156.762       1
13127.587       1
Name: total_income, Length: 19349, dtype: int64


In [149]:
(df.value_counts(subset = ['income_type'])/df.value_counts(subset = ['income_type']).sum()).map("{:.1%}".format) # Checking distribution

income_type                
employee                       51.7%
business                       23.6%
retiree                        17.9%
civil servant                   6.8%
entrepreneur                    0.0%
unemployed                      0.0%
paternity / maternity leave     0.0%
student                         0.0%
dtype: object

We were counted the share of missing values in columns relative to the total number of values in that columns.     
Missing values can be caused by hardware or human error and is random and patterns nature.

In [150]:
# Checking the distribution in the whole dataset
print('Nan dataframe. Income type distribution:')
display(df[df.total_income.isna()]['income_type'].value_counts())

print('Not Nan dataframe. Income type distribution:')
display(df[~df.total_income.isna()]['income_type'].value_counts())

Nan dataframe. Income type distribution:


employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

Not Nan dataframe. Income type distribution:


employee                       10014
business                        4577
retiree                         3443
civil servant                   1312
unemployed                         2
student                            1
entrepreneur                       1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [151]:
print('Nan dataframe. Income type normalized distribution:')
display(df[df.total_income.isna()]['income_type'].value_counts(normalize=True))

print('Not Nan dataframe. Income type normalized distribution:')
display(df[~df.total_income.isna()]['income_type'].value_counts(normalize=True))

Nan dataframe. Income type normalized distribution:


employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64

Not Nan dataframe. Income type normalized distribution:


employee                       0.517493
business                       0.236525
retiree                        0.177924
civil servant                  0.067800
unemployed                     0.000103
student                        0.000052
entrepreneur                   0.000052
paternity / maternity leave    0.000052
Name: income_type, dtype: float64

Original dataset distribution different to the filtered table. It mean we have missing values which need to handle.
We have no any pattern here. Need to continue proceed to handle of missing values.

In [152]:
df_filt = df[(df.days_employed.isna())&(df.total_income.isna())]
(df_filt.value_counts(subset = ['purpose'])/df_filt.value_counts(subset = ['purpose']).sum()).map("{:.1%}".format)

purpose                                 
having a wedding                            4.2%
to have a wedding                           3.7%
wedding ceremony                            3.5%
construction of own property                3.4%
housing transactions                        3.4%
buy real estate                             3.3%
transactions with my real estate            3.3%
purchase of the house for my family         3.3%
transactions with commercial real estate    3.2%
housing renovation                          3.2%
buy commercial real estate                  3.1%
buying property for renting out             3.0%
property                                    2.9%
buy residential real estate                 2.8%
real estate transactions                    2.8%
housing                                     2.8%
building a property                         2.7%
cars                                        2.6%
going to university                         2.6%
to become educated          

## Data transformation



In [153]:
# Let's see all values in education column to check if and what spellings will need to be fixed
df['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [154]:
# Fix the registers if required
df['education']=df['education'].str.lower()

In [155]:
# Checking all the values in the column to make sure we fixed them
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [156]:
# Let's see the distribution of values in the `children` column
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [157]:
# fix the data based on your decision
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)

In [158]:
# Checking the `children` column again to make sure it's all fixed
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [159]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
df['days_employed'].value_counts()

-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: days_employed, Length: 19351, dtype: int64

In [160]:
# Address the problematic values, if they exist
df['days_employed'] = df['days_employed'].apply(lambda x:abs(x))

In [161]:
# Check the result - make sure it's fixed
df['days_employed'].head(10)

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
Name: days_employed, dtype: float64

In [162]:
# Check the `dob_years` for suspicious values and count the percentage
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [163]:
# Address the issues in the `dob_years` column, if they exist
df[df['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,housing
20462,0,338734.868540,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house
20577,0,331741.271455,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property
21179,2,108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate


In [164]:
# Check the result - make sure it's fixed
df.drop(df[df['dob_years']==0].index, inplace = True)

In [165]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [166]:
# Let's see the values for the column
df['family_status'].value_counts()

married              12331
civil partnership     4156
unmarried             2797
divorced              1185
widow / widower        955
Name: family_status, dtype: int64

In [167]:
# Let's see the values in the column
df['gender'].value_counts()

F      14164
M       7259
XNA        1
Name: gender, dtype: int64

In [168]:
# Address the problematic values, if they exist
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


In [169]:
# Check the result - make sure it's fixed
df['gender'] = df['gender'].replace('XNA', 'third gender or unknow')
df['gender'].value_counts()

F                         14164
M                          7259
third gender or unknow        1
Name: gender, dtype: int64

In [170]:
# Let's see the values in the column
df['income_type'].value_counts()

employee                       11064
business                        5065
retiree                         3836
civil servant                   1453
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

Will check duplicates, and if found it will drop it.
                             

In [171]:
# Checking duplicates
df.duplicated().sum()

71

In [172]:
# Address the duplicates, if they exist
duplicateRows = df[df.duplicated()]
print(duplicateRows)

       children  days_employed  dob_years            education  education_id  \
2849          0            NaN         41  secondary education             1   
3290          0            NaN         58  secondary education             1   
4182          1            NaN         34    bachelor's degree             0   
4851          0            NaN         60  secondary education             1   
5557          0            NaN         58  secondary education             1   
...         ...            ...        ...                  ...           ...   
20702         0            NaN         64  secondary education             1   
21032         0            NaN         60  secondary education             1   
21132         0            NaN         47  secondary education             1   
21281         1            NaN         30    bachelor's degree             0   
21415         0            NaN         54  secondary education             1   

           family_status  family_status

In [173]:
df = df.drop_duplicates().reset_index(drop=True)

In [174]:
# Check the size of the dataset that you now have after your first manipulations with it
df.duplicated().sum()

0

In [175]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21353.000000,19260.000000,21353.000000,21353.000000,21353.000000,21353.000000,19260.000000
mean,0.480495,66902.437877,43.475905,0.817777,0.972650,0.081160,26794.435857
std,0.755835,139022.196744,12.242316,0.548764,1.420987,0.273086,16493.972601
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,926.851509,33.000000,1.000000,0.000000,0.000000,16494.872500
50%,0.000000,2197.359790,43.000000,1.000000,0.000000,0.000000,23201.873500
75%,1.000000,5539.681747,53.000000,1.000000,1.000000,0.000000,32539.207750
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


We've removed all duplicates and incorrect values. Now will continued handle for missing values.

# Working with missing values

In [176]:
# Find the dictionaries
#Education dictionary
education_dic = df.set_index('education_id')['education'].to_dict()

#Family status dictionary
family_dic = df.set_index('family_status_id')['family_status'].to_dict()

education_dic, family_dic

({0: "bachelor's degree",
  1: 'secondary education',
  2: 'some college',
  3: 'primary education',
  4: 'graduate degree'},
 {0: 'married',
  1: 'civil partnership',
  2: 'widow / widower',
  3: 'divorced',
  4: 'unmarried'})

### Restoring missing values in `total_income`

There's 'total_income' and 'days_employed' columns with missing values. Will fixed this via create new columns and replase missing values to mean() or median().

In [177]:
df.total_income.describe()

count     19260.000000
mean      26794.435857
std       16493.972601
min        3306.762000
25%       16494.872500
50%       23201.873500
75%       32539.207750
max      362496.645000
Name: total_income, dtype: float64

In [178]:
df[df.total_income.isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21317,2,NaN,47,secondary education,1,married,0,M,business,0,NaN,purchase of a car
21323,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21325,0,NaN,48,bachelor's degree,0,married,0,F,business,0,NaN,building a property
21330,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [179]:
# Let's write a function that calculates the age category
def age_cat(age):
    if age <= 19:
        return 'up to 19'
    elif 20 <= age <= 29:
        return '20-29'
    elif 30 <= age <= 39:
        return '30-39'
    elif 40 <= age <= 49:
        return '40-49'
    elif 50 <= age <= 59:
        return '50-59'
    elif 60 <= age <= 69:
        return '60-69'
    else:
        return 'retired'

df['age_cat'] = df['dob_years'].apply(age_cat)                                        

In [180]:
# Test if the function works
df['age_cat'].value_counts()

30-39       5662
40-49       5354
50-59       4657
20-29       3166
60-69       2331
retired      169
up to 19      14
Name: age_cat, dtype: int64

In [181]:
# Creating new column based on function

df_fill = df.groupby(['age_cat'])['total_income'].mean()
df_fill_dictionary = pd.Series(df_fill).to_dict()

df['total_income_1'] = df['total_income'].fillna(df.age_cat.map(df_fill_dictionary))

In [182]:
df.total_income_1 # Checking how values in the new column



0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21348    35966.698
21349    24959.969
21350    14347.610
21351    39054.888
21352    13127.587
Name: total_income_1, Length: 21353, dtype: float64

In [183]:
# Create a table without missing values and print a few of its rows to make sure it looks fine
df.dropna().head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_cat,total_income_1
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,17932.802
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,42820.568
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,25378.572
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29,40922.170
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49,38484.156
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59,21731.829
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39,15337.093
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49,23108.150


In [184]:
# Look at the mean values for income based on your identified factors
df.total_income_1.mean()

26795.08782838827

In [185]:
# Look at the median values for income based on your identified factors
df.total_income_1.median()

24563.65

In [186]:
#  Write a function that we will use for filling in missing values
def replace_mean(column_replace, column_education, column_family_status, column_gender):
    df[column_replace] = df[column_replace].fillna(df.groupby([
         column_education, column_family_status, column_gender])[column_replace].transform('mean'))
     
df_fill = df.groupby(['dob_years', 'income_type', 'education'])['total_income'].mean()
df_fill_dictionary = pd.Series(df_fill).to_dict()

df['total_income'] = df['total_income'].fillna(df.dob_years.map(df_fill_dictionary))

In [187]:
# Check if it works
replace_mean('total_income', 'dob_years', 'income_type', 'education')
df.loc[12]

children                              0
days_employed                       NaN
dob_years                            65
education           secondary education
education_id                          1
family_status         civil partnership
family_status_id                      1
gender                                M
income_type                     retiree
debt                                  0
total_income               22373.505981
purpose               to have a wedding
age_cat                           60-69
total_income_1             23242.812818
Name: 12, dtype: object

In [188]:
# Check if we got any errors
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21353 entries, 0 to 21352
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21353 non-null  int64  
 1   days_employed     19260 non-null  float64
 2   dob_years         21353 non-null  int64  
 3   education         21353 non-null  object 
 4   education_id      21353 non-null  int64  
 5   family_status     21353 non-null  object 
 6   family_status_id  21353 non-null  int64  
 7   gender            21353 non-null  object 
 8   income_type       21353 non-null  object 
 9   debt              21353 non-null  int64  
 10  total_income      21339 non-null  float64
 11  purpose           21353 non-null  object 
 12  age_cat           21353 non-null  object 
 13  total_income_1    21353 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB


In [189]:
# Replacing missing values 
df[df.total_income.isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_cat,total_income_1
1296,1,NaN,70,primary education,3,civil partnership,1,F,employee,0,NaN,transactions with commercial real estate,retired,20125.658331
1512,0,NaN,53,primary education,3,married,0,F,employee,0,NaN,transactions with my real estate,50-59,25811.700327
3592,0,NaN,40,bachelor's degree,0,married,0,M,retiree,0,NaN,housing renovation,40-49,28551.375635
3599,0,NaN,24,secondary education,1,married,0,F,retiree,0,NaN,purchase of my own house,20-29,25572.630177
3870,1,NaN,68,some college,2,divorced,3,F,employee,0,NaN,building a property,60-69,23242.812818
5502,0,NaN,23,bachelor's degree,0,unmarried,4,M,civil servant,0,NaN,purchase of the house for my family,20-29,25572.630177
5907,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,50-59,25811.700327
8095,0,NaN,64,primary education,3,civil partnership,1,F,civil servant,0,NaN,to have a wedding,60-69,23242.812818
15971,0,NaN,49,some college,2,civil partnership,1,M,business,0,NaN,having a wedding,40-49,28551.375635
16887,0,NaN,39,primary education,3,married,0,F,civil servant,0,NaN,transactions with commercial real estate,30-39,28312.479963


In [190]:
# Checking the number of entries in the columns
df.total_income.fillna(df.total_income_1, inplace=True)

In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21353 entries, 0 to 21352
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21353 non-null  int64  
 1   days_employed     19260 non-null  float64
 2   dob_years         21353 non-null  int64  
 3   education         21353 non-null  object 
 4   education_id      21353 non-null  int64  
 5   family_status     21353 non-null  object 
 6   family_status_id  21353 non-null  int64  
 7   gender            21353 non-null  object 
 8   income_type       21353 non-null  object 
 9   debt              21353 non-null  int64  
 10  total_income      21353 non-null  float64
 11  purpose           21353 non-null  object 
 12  age_cat           21353 non-null  object 
 13  total_income_1    21353 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB


###  Restoring values in `days_employed`

In [192]:
# Distribution of `days_employed` medians based on your identified parameters
df.days_employed.median()

2197.359789516353

In [193]:
# Distribution of `days_employed` means based on your identified parameters
df.days_employed.mean()

66902.437877033

We'll use median() method, cause the data distribution is too unsymmetrical and median give more objective value.

In [194]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter
df_mediana=df.dropna()
df_mediana.groupby(['age_cat', 'income_type']).agg({
    'total_income':['mean', 'median']}).sort_values(by=('total_income','median'), ascending = False)

total_income            
                                              mean      median
age_cat  income_type                                          
20-29    entrepreneur                 79866.103000  79866.1030
40-49    unemployed                   32435.602000  32435.6020
60-69    business                     32494.918350  29171.9890
30-39    business                     33145.949183  28765.5935
40-49    business                     33989.836505  28698.4665
retired  business                     27766.307200  28138.8950
50-59    business                     32385.032725  27299.6345
20-29    business                     28768.323258  25631.8930
40-49    civil servant                28568.272491  24890.7590
30-39    civil servant                27921.836553  24662.7440
retired  employee                     26672.382429  24660.9010
         civil servant                32189.795667  24525.2240
20-29    civil servant                25206.966324  23611.6970
60-69    civil servant                29305.166039  23390.0570
         employee                     27307.606610  23316.9650
30-39    employee                     26191.716908  23218.8030
50-59    civil servant                25838.105730  23136.3020
40-49    employee                     26193.926281  23108.1500
50-59    employee                     26073.759931  22547.8310
40-49    retiree                      27020.126339  22498.7080
20-29    employee                     24125.462244  21588.7985
50-59    retiree                      22221.765833  19526.8120
up to 19 business                     19197.349000  19321.4450
30-39    retiree                      23122.709862  18735.7160
60-69    retiree                      21544.426743  18446.1435
retired  retiree                      18994.044264  17650.4660
20-29    student                      15712.260000  15712.2600
up to 19 employee                     14882.764600  14575.7170
20-29    retiree                      14888.651857  12807.0710
up to 19 civil servant                12125.986000  12125.9860
30-39    unemployed                    9593.119000   9593.1190
         paternity / maternity leave   8612.661000   8612.6610

In [195]:
# Apply function to the income_type
def replace_median(total_income, days_employed):
    df[total_income] = df[total_income].fillna(df.groupby(days_employed)[total_income].transform('median'))
    
replace_median('days_employed', 'age_cat')

In [196]:
# Check if function worked
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21353 entries, 0 to 21352
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21353 non-null  int64  
 1   days_employed     21353 non-null  float64
 2   dob_years         21353 non-null  int64  
 3   education         21353 non-null  object 
 4   education_id      21353 non-null  int64  
 5   family_status     21353 non-null  object 
 6   family_status_id  21353 non-null  int64  
 7   gender            21353 non-null  object 
 8   income_type       21353 non-null  object 
 9   debt              21353 non-null  int64  
 10  total_income      21353 non-null  float64
 11  purpose           21353 non-null  object 
 12  age_cat           21353 non-null  object 
 13  total_income_1    21353 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB


In [197]:
df[(df.days_employed<20000)|((df.days_employed.isna()))]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_cat,total_income_1
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,17932.802
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,42820.568
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29,40922.170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21347,1,2351.431934,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate,30-39,18551.846
21348,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-49,35966.698
21350,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,30-39,14347.610
21351,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30-39,39054.888


In [198]:
# Check the entries in all columns - make sure we fixed all missing values
df.count()

children            21353
days_employed       21353
dob_years           21353
education           21353
education_id        21353
family_status       21353
family_status_id    21353
gender              21353
income_type         21353
debt                21353
total_income        21353
purpose             21353
age_cat             21353
total_income_1      21353
dtype: int64

In [199]:
df[df.days_employed.isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_cat,total_income_1


In [200]:
df.days_employed.describe()

count     21353.000000
mean      64634.317948
std      136895.677023
min          24.141633
25%        1005.629955
50%        2111.489906
75%        5125.978707
max      401755.400475
Name: days_employed, dtype: float64

## Categorization of data



Let's categorize our data. We need to received data's from 4 main sides, particularly it's mumber of children, marital status groups, income type groups and loan purpose groups. 

In [201]:
# Print the values for your selected data for categorization
df_default = df[df['debt']==1]
df_default.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1733 entries, 14 to 21351
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          1733 non-null   int64  
 1   days_employed     1733 non-null   float64
 2   dob_years         1733 non-null   int64  
 3   education         1733 non-null   object 
 4   education_id      1733 non-null   int64  
 5   family_status     1733 non-null   object 
 6   family_status_id  1733 non-null   int64  
 7   gender            1733 non-null   object 
 8   income_type       1733 non-null   object 
 9   debt              1733 non-null   int64  
 10  total_income      1733 non-null   float64
 11  purpose           1733 non-null   object 
 12  age_cat           1733 non-null   object 
 13  total_income_1    1733 non-null   float64
dtypes: float64(3), int64(5), object(6)
memory usage: 203.1+ KB


In [202]:
# Check the unique values
df_default['children'].unique()

array([0, 1, 2, 3, 4])

In [203]:
# Let's write a function to categorize the data based on common topics
def children_cat(qty):
    if qty == 0:
        return 'No' 
    if qty <3:
        return '1-2'
    return '3 and more' 

In [204]:
import warnings
warnings.filterwarnings('ignore')

In [205]:
# Create a column with the categories and count the values for them
df_default.loc[:,'children_cat'] = df_default['children'].apply(children_cat)
df_default['children_cat'].value_counts()

No            1058
1-2            644
3 and more      31
Name: children_cat, dtype: int64

In [206]:
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [207]:
def func_purpose(row):
    try:
        if 'wed' in row:
            return 'wedding'
        elif 'estat' in row or 'hous' in row:
            return 'real_estate'
        elif 'car' in row:
            return 'car'
        else:
            return 'education'
    except:
        return 'no_category'
    
    
# Check the unique values after we apply the function to a column:    
df['purpose'].apply(func_purpose).unique() 

array(['real_estate', 'car', 'education', 'wedding'], dtype=object)

In [208]:
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [209]:
df['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [210]:
df['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [211]:
df.value_counts(subset = ['income_type'])

income_type                
employee                       11029
business                        5058
retiree                         3809
civil servant                   1451
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
dtype: int64

In [212]:
income_dic = {'business':'entrepreneur',
              'unemployed':['paternity / maternity leave', 'student']
}

In [213]:
df.value_counts(subset = ['income_type'])

income_type                
employee                       11029
business                        5058
retiree                         3809
civil servant                   1451
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
dtype: int64

In [214]:
def purpose_cat(goal):
    if 'hous' in goal:
        return 'real estate' 
    elif 'wedding' in goal:
        return 'wedding'
    elif 'car' in goal:
        return 'car'
    else: return 'education'

In [215]:
# Creating column with categories
df.loc[:,'purpose_cat'] = df['purpose'].apply(purpose_cat)
df['purpose_cat'].value_counts()

education      10972
car             4284
real estate     3787
wedding         2310
Name: purpose_cat, dtype: int64

In [216]:
print('education' in 'education')

print('education' in 'for education')
 
print(('edu' in 'university') or ('university' in 'university') or ('edu' in 'for educational purposes'))

True
True
True


In [217]:
df_default['family_status'].unique()

array(['civil partnership', 'unmarried', 'married', 'widow / widower',
       'divorced'], dtype=object)

In [218]:
def family_cat(status):
    if status == 'civil partnership' or status =='married':
        return 'couple' 
    else: return 'single'

In [219]:
df_default.loc[:, 'family_cat'] = df_default['family_status'].apply(family_cat)
df_default['family_cat'].value_counts()

couple    1313
single     420
Name: family_cat, dtype: int64

In [220]:
# Looking through the numerical data in `total income` column for categorization
print(df_default['total_income'].max())
print('----------------')
print(df_default['total_income'].min())
print('----------------')
print(df_default['total_income'].mean())
print('----------------')
print(df_default['total_income'].median())
print('----------------')

352136.354
----------------
3306.762
----------------
26050.377757048354
----------------
23432.859
----------------


In [221]:
print(df_default['total_income'].value_counts().sum())
print('----------------')
print(df_default.loc[df['total_income']>50000].shape[0])
print('----------------')
print(df_default.loc[df['total_income']<10000].shape[0])
print('----------------')

1733
----------------
92
----------------
58
----------------


In [222]:
# Creating function for categorizing into different numerical groups based on ranges
def income_cat(qty):
    if qty < 10000:
        return 'low'
    elif qty <15000:
        return 'low-medium'
    elif qty <25000:
        return 'medium'
    elif qty <35000:
        return 'high-medium'
    elif qty <50000:
        return 'high'
    else:
        return 'high+'

In [223]:
# Creating column with categories
df_default.loc[:, 'income_cat'] = df_default['total_income'].apply(income_cat)

In [224]:
# Count each categories values to see the distribution
(df_default['income_cat'].value_counts()/df_default['income_cat'].value_counts().sum()).map("{:.1%}".format)

medium         40.2%
high-medium    25.3%
low-medium     13.8%
high           12.1%
high+           5.3%
low             3.3%
Name: income_cat, dtype: object

Our total_income groups gave us distribution.

## Checking the Hypotheses


In [225]:
# Check the children data and paying back on time

#Creating `children_cat` column in the whole dataFrame

df['children_cat'] = df['children'].apply(children_cat)

# Creating a pivot table and finding the share of fefaulted people to non defaulted people in the same age group
pivot_children = df.pivot_table(
    index = 'children_cat',
    columns = 'debt',
    values = 'days_employed',
    aggfunc = 'count'
)
pivot_children['ratio'] = (pivot_children[1] / (pivot_children[1]+pivot_children[0]))
pivot_children

#Creating a dictionary of the default rate
children_cat_dict = pd.Series(pivot_children['ratio']).to_dict()

# Calculating default-rate based on the number of children
df['children_def_r'] = df['children_cat'].map(children_cat_dict)
df[['children','children_def_r']].head(10)


debt,0,1,ratio
children_cat,,,
1-2,6309,644,0.092622
3 and more,347,31,0.082011
No,12964,1058,0.075453


,children,children_def_r
0,1,0.092622
1,1,0.092622
2,0,0.075453
3,3,0.082011
4,0,0.075453
5,0,0.075453
6,0,0.075453
7,0,0.075453
8,2,0.092622
9,0,0.075453


### First Conclusion
    
We have result ranking of default risk which crucial depending from the number of children . So thereby, hypothese is doesnt enough clear,  because the people who have 1 or 2 children are a more likely not to pay on time,  while people who have no children are less likely to default.                             

In [226]:
df.groupby('children_cat')['debt'].agg(['count', 'sum', 'mean'])

,count,sum,mean
children_cat,,,
1-2,6953,644,0.092622
3 and more,378,31,0.082011
No,14022,1058,0.075453


In [227]:
def my_mean(x): return x.mean()*100

# We can you our own function in agg method
df.groupby('children_cat')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children_cat,,,
1-2,6953,644,9.262189
3 and more,378,31,8.201058
No,14022,1058,7.545286


In [228]:
def my_mean(x): return '{:.2%} '.format(x.mean())

df.groupby('children_cat')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children_cat,,,
1-2,6953,644,9.26%
3 and more,378,31,8.20%
No,14022,1058,7.55%


In [229]:
# Check the family and paying back on time

#Creating `family_cat` column in the whole dataFrame

# Creating a pivot table and finding the share of fefaulted people to non defaulted people in the same marital status
pivot_family = df.pivot_table(
    index = 'family_status',
    columns = 'debt',
    values = 'days_employed',
    aggfunc = 'count'
)
pivot_family['ratio'] = (pivot_family[1] / (pivot_family[1]+pivot_family[0]))
pivot_family

#Creating a dictionary of the default rate
family_status_dict = pd.Series(pivot_family['ratio']).to_dict()

# Calculating default-rate based on the marital status
df['family_def_r'] = df['family_status'].map(family_status_dict)
df[['family_status','family_def_r']].head(10)


debt,0,1,ratio
family_status,,,
civil partnership,3744,386,0.093462
divorced,1100,85,0.071730
married,11363,927,0.075427
unmarried,2521,273,0.097709
widow / widower,892,62,0.064990


,family_status,family_def_r
0,married,0.075427
1,married,0.075427
2,married,0.075427
3,married,0.075427
4,civil partnership,0.093462
5,civil partnership,0.093462
6,married,0.075427
7,married,0.075427
8,civil partnership,0.093462
9,married,0.075427


### Second Conclusion
    
We have incorrect hypothesis, because 2 categories have more risk to default and these categories are mutually contradictory. Most risky is unmarried group with 9.77% coef, then civil partnership with 9.34%. Widow(er) customers have lowest coef of default risk.

In [230]:
dict(zip(df['family_status_id'],df['family_status']))

{0: 'married',
 1: 'civil partnership',
 2: 'widow / widower',
 3: 'divorced',
 4: 'unmarried'}

In [231]:
# Create a dictionary
family_dict = df[['family_status_id', 'family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
family_dict

def my_mean(x): return '{:.2%} '.format(x.mean())

# Table as in the example above
a = df.groupby('family_status_id')['debt'].agg(['count', 'sum', my_mean])
display(a)

# Replacement
a.reset_index().replace({'family_status_id': family_dict.family_status.to_dict()})

,family_status_id,family_status
0,0,married
1,1,civil partnership
2,2,widow / widower
3,3,divorced
4,4,unmarried


,count,sum,my_mean
family_status_id,,,
0,12290,927,7.54%
1,4130,386,9.35%
2,954,62,6.50%
3,1185,85,7.17%
4,2794,273,9.77%


,family_status_id,count,sum,my_mean
0,married,12290,927,7.54%
1,civil partnership,4130,386,9.35%
2,widow / widower,954,62,6.50%
3,divorced,1185,85,7.17%
4,unmarried,2794,273,9.77%


In [232]:
# Check the income level and paying back on time

#Creating `income_cat` column in the whole dataFrame
df['income_cat']= df['total_income'].apply(income_cat)

# Creating a pivot table and finding the share of fefaulted people to non defaulted people in the same income category
pivot_income = df.pivot_table(
    index = 'income_cat',
    columns = 'debt',
    values = 'days_employed',
    aggfunc = 'count'
)
pivot_income['ratio'] = (pivot_income[1] / (pivot_income[1]+pivot_income[0]))
pivot_income

#Creating a dictionary of the default rate
income_cat_dict = pd.Series(pivot_income['ratio']).to_dict()

# Calculating default-rate based on the total income
df['income_def_r'] = df['income_cat'].map(income_cat_dict)
df[['total_income','income_def_r']].head(10)

debt,0,1,ratio
income_cat,,,
high,2736,209,0.070968
high+,1228,92,0.069697
high-medium,4934,438,0.081534
low,864,58,0.062907
low-medium,2565,239,0.085235
medium,7293,697,0.087234


,total_income,income_def_r
0,40620.102,0.070968
1,17932.802,0.087234
2,23341.752,0.087234
3,42820.568,0.070968
4,25378.572,0.081534
5,40922.170,0.070968
6,38484.156,0.070968
7,21731.829,0.087234
8,15337.093,0.087234
9,23108.150,0.087234


### Third Conclusion
    
The groups medium segment (medium, high-medium and low-medium) are still most risky category from 8.15% up to 8.72%, whereas customers from low segment still are island of stability. Thus, this hypothesis is incorrect too. 

In [233]:
# Check the percentages for default rate for each credit purpose and analyze them

#Creating `purpose_cat` column in the whole dataFrame
df['purpose_cat']= df['purpose'].apply(purpose_cat)

# Creating a pivot table and finding the share of fefaulted people to non defaulted people in the same credit purpose category
pivot_purpose = df.pivot_table(
    index = 'purpose_cat',
    columns = 'debt',
    values = 'days_employed',
    aggfunc = 'count'
)
pivot_purpose['ratio'] = (pivot_purpose[1] / (pivot_purpose[1]+pivot_purpose[0]))
pivot_purpose

#Creating a dictionary of the default rate
purpose_cat_dict = pd.Series(pivot_purpose['ratio']).to_dict()

# Calculating default-rate based on the credit purpose
df['purpose_def_r'] = df['purpose_cat'].map(purpose_cat_dict)
df[['purpose','purpose_def_r']].head(10)



debt,0,1,ratio
purpose_cat,,,
car,3884,400,0.093371
education,10077,895,0.081571
real estate,3533,254,0.067072
wedding,2126,184,0.079654


,purpose,purpose_def_r
0,purchase of the house,0.067072
1,car purchase,0.093371
2,purchase of the house,0.067072
3,supplementary education,0.081571
4,to have a wedding,0.079654
5,purchase of the house,0.067072
6,housing transactions,0.067072
7,education,0.081571
8,having a wedding,0.079654
9,purchase of the house for my family,0.067072


### Fourth Conclusion
    
This hypothese is completely incorrect. The members of group who took loan as buy a car  purpose aswell as for education are high likely to default. Those people who used loan for real estate or to wedding are less the risk of future default.  

## General Conclusion  
    
Our check based on the 4 hypotheses:

- We have result ranking of default risk which crucial depending from the number of children . So thereby, hypothese is not clear, because the people who have no children are a less likely not to pay on time, while people who have 1-2 children are high likely to default.
- 2 categories have more risk to default and these categories are mutually contradictory. Most risky is unmarried group with 9.77% coef, then civil partnership with 9.34%. Widow(er) customers have lowest coef of default risk. While widow people having stability of their obligations fulfilling with totl coef of risk 6.49%. And this is incorrect hypothese.
- People who have low income are less likely to default. And the groups of medium segment (medium, high-medium and low-medium) are most risky category from 8.15% up to 8.72% coef. This hypothese is incorrect.
- Loan purpose has influence on the probability to default. 
The firth hypothese is incorrect either. The members of group who took loan as buy a car purpose aswell as for education are high likely to default. Those people who used loan for real estate or to wedding are less the risk of future default.

                              
As result, we can noted our suggestions were not true, but we made conclusions based on the data and we have proved this.